### Load folders by patient #

In [ ]:
# # By patient
# data_main = r'D:\Baylor\Patient12' # data_main is patient level
# filesDepth = glob.glob(data_main+'\*\*\*\*')
# dirsDepth = filter(lambda f: os.path.isdir(f), filesDepth)

### Or load motion and no motion list

In [1]:
data_main = r"C:\Users\Valued Customer\Desktop"
import csv
import pandas as pd
motionloc = data_main+'/motion_loc.csv'
nomotionloc = data_main+'/nomotion_loc.csv'
motion_addr = pd.read_csv(motionloc, header=None,skip_blank_lines=True) # load whole table, row 22 is NA
nomotion_addr = pd.read_csv(nomotionloc, header=None,skip_blank_lines=True)
dirsmotion = motion_addr.ix[1:,1] # 2nd column is folder address 
dirsnomotion = nomotion_addr.ix[1:,1]
# dirsmotion = dirsmotion[:35] # just take baylor data for now
# dirsnomotion = dirsnomotion[:50] # just take baylor data for now

In [ ]:
# with open(data_main+'/motions.csv', 'r') as f:
#   reader = csv.reader(f)
#   dirs = list(reader) # list of all the new files
# dirsmotion = dirs [1:] # skip header
# with open(data_main+'/nomotion.csv', 'r') as f:
#   reader = csv.reader(f)
#   dirs = list(reader) # list of all the new files
# dirsnomotion = dirs [1:] # skip header

In [2]:
global N_shrink,N_frame
N_shrink=N_frame = 5

### Load Packages

In [3]:
from PIL import Image
from skimage.util import img_as_float
from skimage import data as dt
import os
import glob
import numpy as np
import pandas as pd
import glob, os.path
from numpy import trapz
import csv

### Define motion image function

In [4]:
def motion_features(imagePath):
    ### Load images
    ### Set directory to image file location
    os.chdir(imagePath)  

    shrink = (slice(0, None, N_shrink), slice(0, None, N_shrink)) # down sample by 10

    file_names=glob.glob('*F855*')[0:400] # load 400 PPG images
    F=[img_as_float(dt.load(imagePath+r'\\'+im_name))[shrink] for im_name in file_names ]

    N_row=F[1].shape[0] # number of rows
    N_col=F[1].shape[1]
    F=np.array(F) # convert from list to 3D image array (400, 209, 282)

    ### Check pixel value range
    data=np.array(F)
    #_=plt.hist(data.flatten())
    ran=data.max()-data.min() # range of data

    ### Calculate frame difference
    #### 5 frames apart
    Diff=[]
    counter=0
    for i in range(0,400-N_frame,N_frame):
        Diff.append(abs(np.array(F[i+N_frame-1]-F[i])).sum())

    # normalize diff by number of pixels per frame, and signal range
    Diff=np.array(Diff)/N_row/N_col/ran
    
    ########################### Diff, calculate 5 features
    area=trapz(Diff, dx=N_frame)
    
    # maximum motion
    mx = max(Diff)
    
    # mean motion
    mn = np.mean(Diff)
    
    # maximum gradient
    Diff2 = Diff[1:]
    grad = Diff2-Diff[:-1]
    maxgrad = max(abs(grad))
    
    # std of motion
    sd = np.std(Diff)
    
    ########################### detrend Diff, calculate 5 features
    Diff=Diff-min(Diff)
    
    # Area under the curve
    # Compute the area using the composite trapezoidal rule.
    # http://stackoverflow.com/questions/13320262/calculating-the-area-under-a-curve-given-a-set-of-coordinates-without-knowing-t
    area_de = trapz(Diff, dx=N_frame)
    
    # maximum motion
    mx_de = max(Diff)
    
    # mean motion
    mn_de = np.mean(Diff)
    
    # maximum gradient
    Diff2 = Diff[1:]
    grad = Diff2-Diff[:-1]
    maxgrad_de = max(abs(grad))
    
    # std of motion
    sd_de = np.std(Diff)
    
    # extract body location
    bl=imagePath.split("\\")[5] #parse imagePath, 5th is body location
    
    
    output_motion = [imagePath,bl,area,mx,mn,maxgrad,sd,area_de,mx_de,mn_de,maxgrad_de,sd_de]
    return output_motion

### Find all images

In [5]:
with open(data_main+r"\motion_feature.csv", 'wb') as outfile: # make sure file is closed after writing
    writer = csv.writer(outfile)
    writer.writerow(["foldername","body loc", "AUC", "max","mean","max frame diff","std","AUC", "max","mean","max frame diff","std"])
    for im in dirsmotion:
        output_motion = motion_features(im)
        writer.writerow(output_motion)

In [6]:
with open(data_main+r"\nomotion_feature.csv", 'wb') as outfile: # make sure file is closed after writing
    writer = csv.writer(outfile)
    writer.writerow(["foldername","body loc","AUC", "max","mean","max frame diff","std","AUC", "max","mean","max frame diff","std"])
    for im in dirsnomotion:
        output_motion = motion_features(im)
        writer.writerow(output_motion)